<a href="https://colab.research.google.com/github/CooperJB710/API_Homework/blob/main/Finance_API_Homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import pandas as pd

In [ ]:
apikey = "SsGDHrb6Gz3Lv1MzQ5BY63rQySjlQG6ha9l9D68P"

In [ ]:
def fetch_basic_quotes(symbols: str, apikey: str):
    """
    Fetch basic quote data for the given symbols (comma-separated).
    Returns the JSON response as a dictionary.
    """
    url = "https://yfapi.net/v6/finance/quote"
    querystring = {"symbols": symbols}

    headers = {
        'x-api-key': apikey
    }

    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    return data


In [ ]:
def fetch_quote_summary(symbol: str, modules: str, apikey: str):
    """
    Fetches modules from the quote summary endpoint for a given symbol.
    `modules` can be a comma-separated string of modules, e.g. "summaryDetail,financialData".
    Returns the JSON response as a dictionary.
    """
    url = f"https://yfapi.net/v11/finance/quoteSummary/{symbol}"
    querystring = {
        "symbol": symbol,
        "modules": modules
    }
    headers = {
        'x-api-key': apikey
    }
    response = requests.get(url, headers=headers, params=querystring)
    return response.json()


In [ ]:
def fetch_trending(apikey: str, region: str = "US"):
    """
    Fetches trending stocks for a specified region (default is US).
    Returns a list of symbols that are currently trending.
    """
    url = f"https://yfapi.net/v1/finance/trending/{region}"
    headers = {
        'x-api-key': apikey
    }
    response = requests.get(url, headers=headers)
    data = response.json()

    # Typically found under data['finance']['result'][0]['quotes']
    # Return them as a list of symbols
    try:
        trending_list = data['finance']['result'][0]['quotes']
        symbols = [item['symbol'] for item in trending_list]
        return symbols
    except KeyError:
        return []

In [ ]:
def task1_basic_stock_data(apikey: str):
    # Prompt user
    symbols = input("Enter two or more stock symbols (comma-separated): ").strip()

    # Fetch the data
    data = fetch_basic_quotes(symbols, apikey)

    # Parse results
    results = data.get('quoteResponse', {}).get('result', [])

    # Display
    print("\n--- Task 1: Basic Stock Data ---")
    for r in results:
        ticker = r.get('symbol', 'N/A')
        name = r.get('longName') or r.get('shortName') or 'N/A'
        price = r.get('regularMarketPrice', 'N/A')

        print(f"Stock Ticker: {ticker}, Company: {name}, Current Market Price: ${price}")


In [ ]:
def task2_1_additional_data(apikey: str):
    symbol = input("Enter a single stock symbol for 52W High/Low and ROA: ").strip()

    # The user can choose modules, but we specifically want summaryDetail + financialData for these items
    modules = "summaryDetail,financialData"
    data = fetch_quote_summary(symbol, modules, apikey)

    # Navigate the JSON to find the pieces we need
    quote_summary = data.get('quoteSummary', {}).get('result', [])
    if not quote_summary:
        print("No data returned. Check symbol or API response.")
        return

    # Usually quote_summary is a list with one element
    info = quote_summary[0]

    # Safely extract items
    summary_detail = info.get('summaryDetail', {})
    fifty_two_week_high = summary_detail.get('fiftyTwoWeekHigh', {}).get('raw', None)
    fifty_two_week_low  = summary_detail.get('fiftyTwoWeekLow', {}).get('raw', None)

    financial_data = info.get('financialData', {})
    return_on_assets = financial_data.get('returnOnAssets', {}).get('raw', None)

    # Create a DataFrame with these values
    df = pd.DataFrame({
        'Symbol': [symbol],
        '52_Week_High': [fifty_two_week_high],
        '52_Week_Low': [fifty_two_week_low],
        'Return_on_Assets': [return_on_assets]
    })

    print("\n--- Task 2.1: 52W High/Low and ROA ---")
    print(df)


In [ ]:
def task2_2_trending_stocks(apikey: str):
    trending_symbols = fetch_trending(apikey, region="US")

    if not trending_symbols:
        print("No trending symbols found or API returned empty.")
        return

    # Let’s limit to, for example, the top 5 for demonstration
    trending_symbols = trending_symbols[:5]

    # Prepare a list to hold the data for each trending stock
    data_list = []

    for sym in trending_symbols:
        # We'll fetch summaryDetail + price in one go if desired
        modules = "summaryDetail,price"
        summary_data = fetch_quote_summary(sym, modules, apikey)
        quote_summary = summary_data.get('quoteSummary', {}).get('result', [])
        if not quote_summary:
            continue

        info = quote_summary[0]
        price_info = info.get('price', {})
        summary_detail = info.get('summaryDetail', {})

        # Extract fields
        name = price_info.get('longName') or price_info.get('shortName') or sym
        current_price = price_info.get('regularMarketPrice', {}).get('raw', None)
        fifty_two_week_high = summary_detail.get('fiftyTwoWeekHigh', {}).get('raw', None)
        fifty_two_week_low  = summary_detail.get('fiftyTwoWeekLow', {}).get('raw', None)

        data_list.append({
            'Symbol': sym,
            'Name': name,
            'CurrentPrice': current_price,
            '52_Week_High': fifty_two_week_high,
            '52_Week_Low': fifty_two_week_low
        })

    # Convert to DataFrame
    df = pd.DataFrame(data_list)

    print("\n--- Task 2.2: Current Trending Stocks (Top 5) ---")
    print(df)


In [ ]:
if __name__ == "__main__":
    # 1) Run Task 1: Basic Stock Data
    task1_basic_stock_data(apikey)

    # 2) Run Task 2.1: Additional Data for a chosen symbol
    task2_1_additional_data(apikey)

    # 3) Run Task 2.2: Current Trending Stocks
    task2_2_trending_stocks(apikey)


Enter two or more stock symbols (comma-separated): AAPL, MSFT

--- Task 1: Basic Stock Data ---
Stock Ticker: AAPL, Company: Apple Inc., Current Market Price: $240.36
Enter a single stock symbol for 52W High/Low and ROA: AAPL

--- Task 2.1: 52W High/Low and ROA ---
  Symbol  52_Week_High  52_Week_Low  Return_on_Assets
0   AAPL         260.1       164.08           0.22519

--- Task 2.2: Current Trending Stocks (Top 5) ---
  Symbol                Name  CurrentPrice  52_Week_High  52_Week_Low
0   NVDA  NVIDIA Corporation        131.28        153.13       75.606
1    CRM    Salesforce, Inc.        307.33        369.00      212.000
2   SNOW      Snowflake Inc.        166.19        233.88      107.130
3   IONQ          IonQ, Inc.         29.93         54.74        6.220
4   BYND   Beyond Meat, Inc.          3.56         12.12        3.300
